https://github.com/alexandrebvd/pytorch-challenge/blob/master/Image%20Classifier%20Project_Resnet152_newflowers.ipynb

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import json
from PIL import Image

In [3]:
####JP
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip install Pillow==5.3.0
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
#import torch

    100% |████████████████████████████████| 2.0MB 14.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


  HTTP error 403 while getting http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
  Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl


In [2]:
####JP
# If version is not 5.3.0, re install and restart
import PIL
print(PIL.PILLOW_VERSION)
# If it shows version three reset runtime and start running from next cell

5.3.0


In [3]:
# Imports here
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models, utils
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
# Necesary to unzip the data file
import zipfile
# If version is not 5.3.0, re install and restart
import PIL
print(PIL.PILLOW_VERSION)


5.3.0


In [0]:
#zipfile.ZipFile('flower_data_short.zip').extractall('flower_data') 

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/01_GitHub/pytorch_challenge')

Load Data

In [0]:
data_dir = 'flower_data' #Balanced dataset through oversampling
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
# TODO: Define your transforms for the training and validation sets

# how many samples per batch to load
batch_size = 16
number_classes = 102
n_workers = 4

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224, scale=(0.25, 1.0)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.02),
                                       #transforms.RandomGrayscale(p=0.2),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform = train_transforms)
validation_dataset = datasets.ImageFolder(valid_dir, transform = validation_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = n_workers)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, num_workers = n_workers)

Labels

In [0]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

Build Classifiers

In [13]:
model = models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:08<00:00, 28080506.21it/s]


In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False



model.fc = nn.Sequential(nn.Linear(2048, 1600),
                         nn.ReLU(),
                         nn.Dropout(0.3),
                         nn.Linear(1600, 102),
                         nn.LogSoftmax(dim=1))

model.class_to_idx = train_dataset.class_to_idx

criterion = nn.NLLLoss()

# Only train the classifier parameters; previous parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.0002)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5, last_epoch=-1)

In [0]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

In [0]:
# number of epochs to train the model
n_epochs = 25

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):
    
    scheduler.step()

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    
    model.train()
    
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if device.type=='cuda':
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

        
    ######################    
    # validate the model #
    ######################
    
    model.eval()
    
    with torch.no_grad():
        class_correct = list(0. for i in range(number_classes))
        class_total = list(0. for i in range(number_classes))

        for data, target in validation_loader:
            # move tensors to GPU if CUDA is available
            if device.type=='cuda':
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)


            # convert output probabilities to predicted class
            _, pred = torch.max(output, 1)
            # compare predictions to true label
            correct = np.squeeze(pred.eq(target.data.view_as(pred)))
            # calculate test accuracy for each object class
            for i in range(int(output.size(0))):
                label = target.data[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1

        #Organizing the list to match the folders in order
        class_correct_ordered = list(0. for i in range(len(class_correct)))
        class_total_ordered = list(0. for i in range(len(class_total)))
        for key, value in model.class_to_idx.items():
            class_correct_ordered[int(key)-1] = class_correct[value]
            class_total_ordered[int(key)-1] = class_total[value]

        # calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(validation_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    for i in range(number_classes):
        if class_total_ordered[i] > 0:
            print('Validation Accuracy of class %3s (%25s): %0.2f%% (%2d/%2d)' % (
                str(i+1), cat_to_name[str(i+1)], 100 * class_correct_ordered[i] / class_total_ordered[i],
                np.sum(class_correct_ordered[i]), np.sum(class_total_ordered[i])))
        else:
            print('Validation Accuracy of class %3s (%25s): N/A (no training examples)' % (str(i+1), cat_to_name[str(i+1)]))
            
    valid_accuracy = 100. * np.sum(class_correct_ordered) / np.sum(class_total_ordered)
    print('\nValidation Accuracy (Overall): %0.2f%% (%2d/%2d)\n' % (
        valid_accuracy, np.sum(class_correct_ordered), np.sum(class_total_ordered)))
    

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...\n\n'.format(
        valid_loss_min, valid_loss))
        
        checkpoint = {'model_state_dict':model.state_dict(),
             #'optimizer_state_dict':optimizer.state_dict(),
             'classifier_model':model.fc,
             'epochs':n_epochs,
             'best_accuracy':valid_accuracy,
             'model_class_idx':model.class_to_idx}
        torch.save(checkpoint, 'model_resnet152_pytorch_challenge.pt')
        
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.001372 	Validation Loss: 0.557788
Validation Accuracy of class   1 (            pink primrose): 37.50% ( 3/ 8)
Validation Accuracy of class   2 (hard-leaved pocket orchid): 100.00% ( 6/ 6)
Validation Accuracy of class   3 (         canterbury bells): 50.00% ( 1/ 2)
Validation Accuracy of class   4 (                sweet pea): 0.00% ( 0/ 6)
Validation Accuracy of class   5 (         english marigold): 100.00% ( 7/ 7)
Validation Accuracy of class   6 (               tiger lily): 100.00% ( 1/ 1)
Validation Accuracy of class   7 (              moon orchid): 0.00% ( 0/ 1)
Validation Accuracy of class   8 (         bird of paradise): 100.00% ( 5/ 5)
Validation Accuracy of class   9 (                monkshood): 66.67% ( 2/ 3)
Validation Accuracy of class  10 (            globe thistle): 100.00% ( 4/ 4)
Validation Accuracy of class  11 (               snapdragon): 80.00% ( 8/10)
Validation Accuracy of class  12 (              colt's foot): 100.00% ( 5/ 5)
Validation 

In [0]:
# TODO: Save the checkpoint (Already done above)
'''checkpoint = {'model_state_dict':model.state_dict(),
             'optimizer_state_dict':optimizer.state_dict(),
             'classifier_model':model.fc,
             'epochs':n_epochs,
             'best accuracy':test_accuracy,
             'model_class_idx':model.class_to_idx}
    torch.save(checkpoint, 'model_pytorch_challenge.pt')'''

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    
    model = models.resnet152(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
    
    model.fc = checkpoint['classifier_model']
    model.class_to_idx = checkpoint['model_class_idx']
    model.load_state_dict(checkpoint['model_state_dict'])
    
    #optimizer = optim.Adam(model.fc.parameters())
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    epochs = checkpoint['epochs']
    best_accuracy = checkpoint['best_accuracy']
    
    return model

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    preprocess = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     normalize])
    im = Image.open(image)
    im_tensor = preprocess(im)

    return im, im_tensor

In [0]:
pil,tensor = process_image('random flower pictures/image_04927.jpg')
tensor.shape
pil

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [0]:

imshow(tensor)

In [0]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

    model.eval()
    
    model.cpu()
    
    # load image as torch.Tensor
    _, image = process_image(image_path)

    # Un-squeeze returns a new tensor with a dimension of size one
    image = image.unsqueeze(0)

    # Disabling gradient calculation
    with torch.no_grad():
        output = model(image)
        top_prob, top_labels = torch.topk(output, topk)

        # Calculate the exponential
        top_prob = top_prob.exp()

    class_to_idx_inv = {model.class_to_idx[key]: int(key) for key, value in model.class_to_idx.items()}
    mapped_classes = []

    for label in top_labels.numpy()[0]:
        mapped_classes.append(class_to_idx_inv[label])

    return top_prob.numpy()[0], mapped_classes

In [0]:
# TODO: Display an image along with the top 5 classes after loading the model
model_loaded = load_checkpoint('model_resnet152_pytorch_challenge.pt')

img_cat = {'random flower pictures/image_04903.jpg':cat_to_name['20'],
           'random flower pictures/image_04927.jpg':cat_to_name['20'],
           'random flower pictures/image_07925.jpg':cat_to_name['100'],
           'random flower pictures/image_08067.jpg':cat_to_name['31'],
           'random flower pictures/image_06905.jpg':cat_to_name['31'],
           'random flower pictures/image_07075.jpg':cat_to_name['67']}

for img, cat in img_cat.items():
    top_prob, top_classes = predict(img, model_loaded)

    label = top_classes[0]

    fig = plt.figure(figsize=(6,6))
    sp_img = plt.subplot2grid((15,9), (0,0), colspan=9, rowspan=9)
    sp_prd = plt.subplot2grid((15,9), (9,2), colspan=5, rowspan=5)

    image = Image.open(img)
    sp_img.axis('off')
    sp_img.set_title(f'{cat}')
    sp_img.imshow(image)

    labels = []
    for class_idx in top_classes:
        labels.append(cat_to_name[str(class_idx)] + ' (class ' + str(class_idx) + ')')

    yp = np.arange(5)
    sp_prd.set_yticks(yp)
    sp_prd.set_yticklabels(labels)
    sp_prd.set_xlabel('Probability')
    sp_prd.invert_yaxis()
    sp_prd.barh(yp, top_prob, xerr=0, align='center', color='blue')

    plt.show()
    print(f'Correct prediction: {cat == cat_to_name[str(label)]}')